In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Parent directory: {parent_dir}")

Parent directory: /u/weim/lob


In [2]:
lots_list = [20,60]
n_samples = 10000
train_iter = 200

In [3]:
# Should update this to include: Dirichlet, Normal+Softmax 
# Note: need more training iterations like 400 (or 500 iterations)
# For 200 iterations, in the noise setting, the rl agent is not better than the noise agent 
# note dirichlet agents also seeems to work well 
lots_list = [20,60]
n_samples = 10000
train_iter = 400
tag = 'GAE'  # could be empty string ''
tag = None
tage = 'long_horizon'
batch_size = 12800


folder_path = f"{parent_dir}/rewards"
data = {}
# agent_list = ['sl_agent', 'linear_sl_agent', 'log_normal', 'dirichlet']
# agent_list = ['sl_agent', 'linear_sl_agent', 'log_normal', 'dirichlet']
agent_list = ['sl_agent', 'linear_sl_agent', 'log_normal', 'dirichlet']
for env in ['noise', 'flow', 'strategic']:
    data[env] = {}
    for lots in lots_list:
        data[env][lots] = {}
        for agent in agent_list:            
            try: 
                if agent in ['log_normal', 'dirichlet']:               
                    # small batch size 
                    # name =  f'{folder_path}/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_{train_iter}_bsize_3200_{agent}_GAE.npz'
                    # medium batch size
                    # name =  f'/u/weim/lob/rewards/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_6400_{agent}.npz'
                    if tag is not None:
                        name =  f'{folder_path}/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_{train_iter}_bsize_{batch_size}_{agent}_{tag}.npz'
                    else:
                        name =  f'{folder_path}/{env}_{lots}_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_{train_iter}_bsize_{batch_size}_{agent}.npz'
                else:
                    # might need to updating the naming here
                    name = f'{folder_path}/{env}_{lots}_episodes_{10000}_eval_seed_100_{agent}.npz'
                print(f'loading from {name}')
                data[env][lots][agent] = np.load(name)['rewards']
            except:
                print(f"path doesnt exist: {name}")
                data[env][lots][agent] = 'no data aviailable'

loading from /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_sl_agent.npz
loading from /u/weim/lob/rewards/noise_20_episodes_10000_eval_seed_100_linear_sl_agent.npz
loading from /u/weim/lob/rewards/noise_20_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_log_normal.npz
loading from /u/weim/lob/rewards/noise_20_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_dirichlet.npz
loading from /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_sl_agent.npz
loading from /u/weim/lob/rewards/noise_60_episodes_10000_eval_seed_100_linear_sl_agent.npz
loading from /u/weim/lob/rewards/noise_60_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_log_normal.npz
loading from /u/weim/lob/rewards/noise_60_seed_0_eval_seed_100_eval_episodes_10000_num_iterations_400_bsize_12800_dirichlet.npz
loading from /u/weim/lob/rewards/flow_20_episodes_10000_eval_seed_100_sl_agent.npz
loading from /u/weim/lob/rewards/flow_20_episodes_100

In [6]:
# medium batch results are not as good as large batch results 
# tag = 'small_batch'
# tag = 'small_batch_GAE'
# tag = 'medium_batch'

folder_path = f"{parent_dir}/latex_tables"
list_of_dfs = []
# n_lots = [20, 60]
# print(filler)
for env in ['noise', 'flow', 'strategic']:
    data_for_df = {}
    name_dict = {'sl_agent': 'SL', 'linear_sl_agent': 'TWAP', 'dirichlet': 'DR', 'log_normal': 'LN', 'log_normal_learn_std': 'LNVAR'}
    # agent_names = ['sl_agent', 'linear_sl_agent', 'dirichlet', 'log_normal']
    agent_names = ['sl_agent', 'linear_sl_agent', 'log_normal', 'dirichlet']
    for agent in agent_names:
        print(agent)
        name = name_dict[agent]
        data_for_df[f'E[{name}]'] = []
        data_for_df[f'Std[{name}]'] = []
        for lots in lots_list:
            try:
                data_for_df[f'Std[{name}]'] += [np.std(data[env][lots][agent])]
                data_for_df[f'E[{name}]'] += [np.mean(data[env][lots][agent])]                
            except:
                print(f"no data available for {env}, {lots}, {agent}")
                data_for_df[f'Std[{name}]'] += [np.nan]
                data_for_df[f'E[{name}]'] += [np.nan]
    df = pd.DataFrame.from_dict(data_for_df).round(2)    
    df.index.name = 'Lots'
    index_name = [f'{env}_{l}' for l in lots_list]
    df.index = index_name
    df = df.round(2)
    list_of_dfs.append(df)

complete_df = pd.concat(list_of_dfs, axis=0)
print(complete_df)


highest_exp = []
lowest_std = []
for idx in complete_df.index:
    # expected_values = complete_df.loc[idx, ['E[SL]', 'E[TWAP]', 'E[DR]', 'E[LN]']]
    expected_values = complete_df.loc[idx, ['E[SL]', 'E[TWAP]', 'E[LN]', 'E[DR]']]
    max_idx = expected_values.idxmax()
    highest_exp.append((idx, max_idx))
    # std_values = complete_df.loc[idx, ['Std[SL]', 'Std[TWAP]', 'Std[DR]', 'Std[LN]']]
    std_values = complete_df.loc[idx, ['Std[SL]', 'Std[TWAP]', 'Std[LN]', 'Std[DR]']]
    min_idx = std_values.idxmin()
    lowest_std.append((idx, min_idx))

complete_df = complete_df.astype(str)
for idx, max_idx in highest_exp:
    complete_df.loc[idx, max_idx] = fr"\textbf{{{complete_df.loc[idx, max_idx]}}}"

for idx, min_idx in lowest_std:
    complete_df.loc[idx, min_idx] = fr"\textbf{{{complete_df.loc[idx, min_idx]}}}"

complete_df = complete_df.drop(['noise', 'flow', 'strategic'], errors='ignore')
complete_df.insert(0, r'\#Lots', complete_df.index.astype(str))
complete_df.iloc[:,0] = [20,60]*3

complete_df.insert(0, r'Market', complete_df.index.astype(str))
complete_df.iloc[:,0] = ['Noise', '', 'Noise \& Tactical', '', 'Noise \& Tactical', '\& Strategic']

rename_dict = {'E[SL]': r'$\mathbb{E}[\text{SL}]$', 'Std[SL]': r'$\sigma[\text{SL}]$', 
               'E[TWAP]': r'$\mathbb{E}[\text{TWAP}]$', 'Std[TWAP]': r'$\sigma[\text{TWAP}]$', 
               'E[LN]': r'$\mathbb{E}[\text{LN}]$', 'Std[LN]': r'$\sigma[\text{LN}]$', 
               'E[DR]': r'$\mathbb{E}[\text{DR}]$', 'Std[DR]': r'$\sigma[\text{DR}]$',
            #    'E[DR]': r'$\mathbb{E}[\text{LNVAR}]$', 'Std[LNVAR]': r'$\sigma[\text{LNVAR}]$'
               }
print(complete_df)
complete_df = complete_df.rename(columns=rename_dict)
complete_df = complete_df.where(pd.notnull(complete_df), '')

latex_table = complete_df.to_latex(float_format="%.2f", index=False, column_format='l'+'c'*9, escape=False)
saving_path = f"{folder_path}/results_table_latest_with_dirichlet_{tag}.tex" if tag is not None else f"{folder_path}/results_table_latest_with_dirichlet.tex"
saving_path = f"{folder_path}/results_table_log_normal_long_horizon.tex" 
tag = '' if tag is None else tag
with open(saving_path, "w") as f:
    content = (
        r"\begin{table}[htpb]"+
        r"\label{table:" + tag +"}"+
        r"\begin{center}"+
        r"    \begin{scriptsize}"+
        r"        \begin{sc}"
        + latex_table +
        r"        \end{sc}"+
        r"    \end{scriptsize}"+
        r"\end{center}"+
        r"\end{table}"        
    )
    f.write(content)

sl_agent
linear_sl_agent
log_normal
dirichlet
sl_agent
linear_sl_agent
log_normal
dirichlet
sl_agent
linear_sl_agent
log_normal
dirichlet
              E[SL]  Std[SL]  E[TWAP]  Std[TWAP]  E[LN]  Std[LN]  E[DR]  \
noise_20       0.52     1.19    -0.06       0.94   0.64     0.88   0.46   
noise_60      -1.09     1.34    -1.40       0.98  -0.81     0.93  -0.79   
flow_20        0.10     1.43     0.48       0.68   0.79     0.67   0.72   
flow_60       -3.36     0.99    -0.96       0.95  -0.24     0.66  -0.34   
strategic_20  -1.64     2.95    -0.36       3.03   1.11     2.23   1.01   
strategic_60  -2.51     3.67    -1.45       3.46   0.11     2.42   0.03   

              Std[DR]  
noise_20         0.91  
noise_60         0.98  
flow_20          0.67  
flow_60          0.66  
strategic_20     2.12  
strategic_60     1.99  
                         Market \#Lots  E[SL] Std[SL] E[TWAP] Std[TWAP]  \
noise_20                  Noise     20   0.52    1.19   -0.06      0.94   
noise_60          